### Grand Theft Data. Парсим Домофонд

тут должно быть необычайно полезное описание, вместо которого будет лишь одна ссылка: 
https://fulyankin.github.io/Parsers/

In [2]:
#библиотеки

from bs4 import BeautifulSoup    # парсинг
import requests                  # читать сайты
import csv                       # записывать данные
import json                      # язык данных
import re                        # regex

In [3]:
head = 'https://www.domofond.ru'
listing = '/arenda-kvartiry-sankt_peterburg-c3414?RentalRate=Month&Page='

Эксперементальным путём выяснили, что страница выдаёт по 27 квартир при общем количестве в 11к+. нам хватит 10.5к:
*10503/27 = 389*.  Посмотрим на структуру одной страницы с 27 квартирами:

In [4]:
response = requests.get(head + listing + '1')
html = response.content
soup = BeautifulSoup(html,'html.parser')
soup


<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/static/images/apple/apple-touch-icon-60x60.png" rel="apple-touch-icon"/>
<link href="/static/images/apple/apple-touch-icon-76x76.png" rel="apple-touch-icon" sizes="76x76"/>
<link href="/static/images/apple/apple-touch-icon-120x120.png" rel="apple-touch-icon" sizes="120x120"/>
<link href="/static/images/apple/apple-touch-icon-152x152.png" rel="apple-touch-icon" sizes="152x152"/>
<!-- Google Tag Manager -->
<noscript><iframe height="0" src="//www.googletagmanager.com/ns.html?id=GTM-MZPV7N" style="display:none;visibility:hidden" width="0"></iframe></noscript>
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
            new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
            j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
            '//www.googletagmanager.com/gtm.js?id='+i+dl;f.parent

Видим, что почти сразу после Google Analytics в начале идёт JSON формат, в котором и забиты все данные. Начинается это примерно с window.INITIAL_DATA. Прогоняем код через [форматтер JSON](http://jsonformatter.curiousconcept.com) и замечаем:
* Путь к ссылкам на кв - **itemState.items.[0-20].itemUrl**
* Путь к листу квартир меняется только в **...&Page=2** (и т.д.)

In [6]:
links = list()

In [53]:
from fake_useragent import UserAgent  # чтобы дольше парсить
UserAgent().firefox #chrome

'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/58.0.1'

In [56]:
# если не прогружает все сслки сразу - делим на интервалы, между которыми свитчаем браузеры чанком выше.

for i in range(70, 140):
    
    response = requests.get(head + listing + str(i+1))
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    links_raw = str(soup.findAll('script')[7])
    links_raw = links_raw.replace('<script>window.__INITIAL_DATA__ = ','')
    links_raw = links_raw.replace('</script>','')
    links_json = json.loads(links_raw)
    
    for j in range(0, 27):
        
        links.append(head + links_json['itemsState']['items'][j]['itemUrl'])
        j += 1

print("Извлечено", len(links), "ссылок")

IndexError: list index out of range

In [58]:
len(links)

11136

In [60]:
import pandas
df = pandas.DataFrame(data={"url": links})
df.to_csv("./file.csv", sep=',',index=False)